In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

Lectura de datos

In [2]:
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv').values.ravel()

In [3]:
X.head()

,country_Azerbaijan,country_Other,country_T.C.,age,sex,station_Fall,station_Spring,station_Summer,station_Winter,fever_temperature,...,chronic_hematologic_disease,aids_hiv,diabetes_mellitus_type_1,diabetes_mellitus_type_2,rheumatologic_disorder,dementia,tuberculosis,smoking,other_risks,previous_positives
0,-0.088303,-0.129387,0.225631,0.233591,0.847138,-0.587958,-0.483181,-0.501048,1.353342,1.117438,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
1,-0.088303,-0.129387,0.225631,-1.092943,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-0.447312,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
2,-0.088303,-0.129387,0.225631,-0.927126,0.847138,-0.587958,-0.483181,-0.501048,1.353342,0.856647,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,0.663176
3,-0.088303,-0.129387,0.225631,0.952130,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-1.099292,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014
4,-0.088303,-0.129387,0.225631,1.615396,0.847138,-0.587958,-0.483181,-0.501048,1.353342,-0.577708,...,-0.042812,-0.038583,-0.041909,-0.042362,-0.039562,-0.039562,-0.038085,-0.314681,-0.033834,-0.405014


División de datos

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y) # stratify=y to keep the same class distribution
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train) # stratify=y to keep the same class distribution

print('Training set:', X_train.shape)
print('Validation set:', X_val.shape)
print('Test set:', X_test.shape)

Training set: (21251, 56)
Validation set: (2362, 56)
Test set: (2624, 56)


# Regresión Logística 

In [48]:
# 1. RFE con Regresión Logística
log_reg = LogisticRegression(class_weight='balanced')
rfe = RFE(estimator=log_reg, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=LogisticRegression(class_weight='balanced'),
    n_features_to_select=15)

In [49]:
# Ver las características seleccionadas
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'station_Fall',
       'station_Spring', 'station_Winter', 'history_of_fever', 'cough',
       'shortness_of_breath', 'conjunctivitis', 'fatigue_malaise', 'diarrhoea',
       'vomiting_nausea', 'smoking', 'previous_positives'],
      dtype='object')


In [ ]:

# Definir los hiperparámetros para GridSearch
param_grid_lr = {
    'C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'penalty': ['l1','l2'],  # Penalización
}

# GridSearchCV para Logistic Regression
grid_search_lr = GridSearchCV(estimator=LogisticRegression(max_iter=1000, solver='liblinear'), param_grid=param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

# Mejor combinación de hiperparámetros
print("Mejores parámetros para Regresión Logística:", grid_search_lr.best_params_)

# Accuracy en el conjunto de validación
print("Accuracy en el conjunto de validación:", grid_search_lr.score(X_val[selected_features], y_val))

# Matriz de confusión en el conjunto de validación
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_lr.predict(X_val[selected_features])))

# Reporte de clasificación en el conjunto de validación
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_lr.predict(X_val[selected_features])))

Mejores parámetros para Regresión Logística: {'C': 1, 'penalty': 'l2'}
Accuracy en el conjunto de validación: 0.8598645215918713
Matriz de confusión en el conjunto de validación:
[[  81  282]
 [  49 1950]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.62      0.22      0.33       363
           1       0.87      0.98      0.92      1999

    accuracy                           0.86      2362
   macro avg       0.75      0.60      0.63      2362
weighted avg       0.84      0.86      0.83      2362



# Random Forest

In [53]:
# RFE con Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=15)

In [54]:
# Ver las características seleccionadas
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_T.C.', 'age', 'sex', 'station_Fall', 'station_Spring',
       'fever_temperature', 'oxygen_saturation', 'history_of_fever', 'cough',
       'sore_throat', 'headache', 'fatigue_malaise', 'muscle_aches',
       'diarrhoea', 'previous_positives'],
      dtype='object')


In [55]:
# Definir los hiperparámetros para GridSearch
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [30, 50, 70],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5],
    'max_features': ['sqrt', None]
}

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

# Mejor combinación de hiperparámetros
print("Mejores parámetros para Random Forest:", grid_search_rf.best_params_)
# Accuracy en el conjunto de validación
print("Accuracy en el conjunto de validación:", grid_search_rf.score(X_val[selected_features], y_val))
# Matriz de confusión en el conjunto de validación
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_rf.predict(X_val[selected_features])))

Mejores parámetros para Random Forest: {'max_depth': 50, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Accuracy en el conjunto de validación: 0.8763759525825572
Matriz de confusión en el conjunto de validación:
[[ 156  207]
 [  85 1914]]


In [56]:
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_rf.predict(X_val[selected_features])))

Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.65      0.43      0.52       363
           1       0.90      0.96      0.93      1999

    accuracy                           0.88      2362
   macro avg       0.77      0.69      0.72      2362
weighted avg       0.86      0.88      0.87      2362



In [8]:
# rfe para svc
from sklearn.svm import SVC
svc = SVC(kernel='linear')
rfe = RFE(estimator=svc, n_features_to_select=15)
rfe.fit(X_train, y_train)

RFE(estimator=SVC(kernel='linear'), n_features_to_select=15)

In [9]:
# Ver las características seleccionadas
selected_features = X_train.columns[rfe.support_]
print("Características seleccionadas con RFE:", selected_features)

Características seleccionadas con RFE: Index(['country_Azerbaijan', 'country_Other', 'country_T.C.', 'sex',
       'station_Spring', 'station_Summer', 'station_Winter',
       'fever_temperature', 'runny_nose', 'shortness_of_breath', 'chest_pain',
       'headache', 'muscle_aches', 'joint_pain', 'vomiting_nausea'],
      dtype='object')


In [10]:
# import svm
from sklearn.svm import SVC

# Definir los hiperparámetros para GridSearch
param_grid_svm = {
    'C': [0.1, 1],  # Parámetro de regularización
    'gamma': [1, 0.1, 0.01],  # Coeficiente del kernel
    'kernel': ['rbf']  # Kernel
}

# GridSearchCV para SVM
grid_search_svm = GridSearchCV(estimator=SVC(class_weight='balanced'), param_grid=param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train[selected_features], y_train)  # Utilizando las características seleccionadas

# Mejor combinación de hiperparámetros
print("Mejores parámetros para SVM:", grid_search_svm.best_params_)

# Accuracy en el conjunto de validación
print("Accuracy en el conjunto de validación:", grid_search_svm.score(X_val[selected_features], y_val))

# Matriz de confusión en el conjunto de validación
print("Matriz de confusión en el conjunto de validación:")
print(confusion_matrix(y_val, grid_search_svm.predict(X_val[selected_features])))

# Reporte de clasificación en el conjunto de validación
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_val, grid_search_svm.predict(X_val[selected_features])))

Mejores parámetros para SVM: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy en el conjunto de validación: 0.7722269263336156
Matriz de confusión en el conjunto de validación:
[[ 217  146]
 [ 392 1607]]
Reporte de clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       0.36      0.60      0.45       363
           1       0.92      0.80      0.86      1999

    accuracy                           0.77      2362
   macro avg       0.64      0.70      0.65      2362
weighted avg       0.83      0.77      0.79      2362

